In [8]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
import pyautogui
from numpy import ones,vstack
from numpy.linalg import lstsq
from statistics import mean
from mss import mss
from directkeys import PressKey, ReleaseKey, W, A, S, D
import pyautogui

In [9]:
# Constants

In [14]:
# Region of Interest
def roi(image, vertices):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, vertices, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def draw_lanes(img, lines, color=[0, 255, 255], thickness=3):

    # we find the line on the screen, which reaches the farthest point, i.e, basically it will reach the horizon and for that the coordinate's y-value will be lesser than the rest
    # we cannot assume that the horizon will be at the same point
    # and lastly we find 2 most horizon-extending lines that gotta be the lanes

    # if this fails, go with some default line
    try:

        # finds the maximum y value for a lane marker 
        # (since we cannot assume the horizon will always be at the same point.)

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1],ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx,i in enumerate(lines):
            for xyxy in i:
                # These four lines:
                # modified from http://stackoverflow.com/questions/21565994/method-to-return-the-equation-of-a-straight-line-given-two-points
                # Used to calculate the definition of a line, given two sets of coords.
                x_coords = (xyxy[0],xyxy[2])
                y_coords = (xyxy[1],xyxy[3])
                A = vstack([x_coords,ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                # Calculating our new, and improved, xs
                x1 = (min_y-b) / m
                x2 = (max_y-b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m,b,line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m,b,line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]      # slope of lane 1
        lane2_id = top_lanes[1][0]      # slope of lane 2
        # we return the above 2 also

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    except Exception as e:
        print(str(e))


# Detecting edges
def process_image(original_image):
    
    # Convert to gray
    processed_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    #edge detection
    processed_image = cv2.Canny(processed_image, threshold1=200, threshold2=300)
    
    processed_image = cv2.GaussianBlur(processed_image, (3, 3), 0)

    vertices = np.array([[10, 500], [10, 300], [300, 200], [500, 200], [800, 300], [800, 500]])   # Changeable
    #vertices = np.array([[100, 500], [100, 300], [300, 200], [500, 200], [700, 300], [700, 500]])
    processed_image = roi(processed_image, [vertices])
    
    # detecting lines
    lines = cv2.HoughLinesP(processed_image, 1, np.pi/180, 180, np.array([]), 100, 5)    # takes edge detect images, returns array of arrays of lines
    
    m1 = 0
    m2 = 0
    try:
        l1, l2, m1, m2 = draw_lanes(original_image,lines)
        cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [255,0,255], 20)
        cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [255,0,255], 20)
    except Exception as e:
        print(str(e))
        pass
    try:
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_image, (coords[0], coords[1]), (coords[2], coords[3]), [255,0,255], 20)
                
                
            except Exception as e:
                print(str(e))
    except Exception as e:
        pass


    return processed_image, original_image, m1, m2

'''
Now the basic logic is, when both the main lines are on the right of the person/car/bike, then you're going extremely right, so go left, and same for left. Basically, when m1 and m2 both -ve, then go left, if both +ve go right, if one +ve and one -ve go straight
'''

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

def left():
    PressKey(A)
    #ReleaseKey(W)#
    ReleaseKey(D)
    #ReleaseKey(A)   # Basically tapping the key 'A' once
    #ReleaseKey(W)

def right():
    PressKey(D)
    #ReleaseKey(W)#
    ReleaseKey(A)
    #ReleaseKey(D)   # Basically tapping the key 'A' once
    #ReleaseKey(W)

def stop():
    ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

In [16]:
for i in list(range(0, 4))[::-1]:
    print(i + 1)
    time.sleep(1)

monitor = {'top' : 40, 'left' : 0, 'width' : 800, 'height' : 600}
sct = mss()
last_time = time.time()

# in this main loop we do precessing
while(True):
    sct_image = sct.grab(monitor)
    screen = np.array(sct_image)
    new_screen, original_image, m1, m2 = process_image(screen)
    print("Capture FPS = " + str(int(1/(time.time() - last_time))))
    last_time = time.time()
    ##cv2.imshow('capture', new_screen)

    if m1 < 0 and m2 < 0:
        right()
    elif m1 > 0 and m2 > 0:
        left()
    else:
        straight()

    cv2.imshow('capture2', original_image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

4
3
2
1
cannot convert float NaN to integer
'NoneType' object is not iterable
Capture FPS = 11
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 7
cannot convert float NaN to integer
'NoneType' object is not iterable
Capture FPS = 9
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 11
cannot convert float NaN to integer
'NoneType' object is not iterable
Capture FPS = 10
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 11
cannot convert float NaN to integer
'NoneType' object is not iterable
Capture FPS = 15
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 13
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 10
cannot convert float infinity to integer
'NoneType' object is not iterable
Capture FPS = 9
cannot convert float NaN to integer
'NoneType' object is not iterable
Capture FPS = 8
cannot co